# Introduction and Background

In this notebook, we introduce kernel-based versions of the models covered in the [Linear Methods](1_PCA_MDS_PCR.ipynb) notebook.

As always, for each model, we first go step-by-step through the derivation, with equations, embedded links, and citations supplied where useful. Second, we will employ a "Sklearn/Skcosmo Class" for the models, which can be found in the skcosmo/sklearn module and contains all necessary functions.

**Note**: To display values of variables in the markdown cells, you should [enable the jupyter contrib nbextension](https://stackoverflow.com/questions/52812231/print-variable-in-jupyter-notebook-markdown-cell-python).

In [ ]:
#!/usr/bin/env python3

import sys

# Maths things
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Local Utilities for Notebook
sys.path.append("../")
from utilities.general import load_variables, sorted_eig, get_stats
from utilities.plotting import (
    plot_projection,
    plot_regression,
    check_mirrors,
    get_cmaps,
    table_from_dict,
)
from sklearn.metrics.pairwise import linear_kernel, rbf_kernel
from sklearn.decomposition import PCA, KernelPCA
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from functools import partial

from skcosmo.decomposition import KernelPCovR
from skcosmo.preprocessing import KernelNormalizer

cmaps = get_cmaps()
plt.style.use("../utilities/kernel_pcovr.mplstyle")
dbl_fig = (2 * plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1])

First, we must load the data. For a step-by-step explanation of this, please see [Importing Data](X_ImportingData.ipynb). As mentioned in the [foreword notebook](0_Foreword.ipynb), pre-computed features should be downloaded from [here](https://www.dropbox.com/s/itokckbbkvxaqsk/precomputed.npz?dl=0) and the file `precomputed.npz` should be copied to the `datasets/` folder.

In [ ]:
var_dict = load_variables()
locals().update(var_dict)

# Constructing Kernels

## The Kernel Trick

Many kernel methods are similar to linear methods,
except that we take advantage of the [**kernel trick**](https://en.wikipedia.org/wiki/Kernel_method#Mathematics:_the_kernel_trick) in order to 
introduce a non-linear transformation of the feature space.

The kernel trick consists in introducing a [**positive definite**](https://en.wikipedia.org/wiki/Positive-definite_kernel) function of pairs of samples, $ k(\mathbf{x}, \mathbf{x}^{\prime})$, that defines implicitly a higher (possibly infinite) dimensional feature space, in which one can apply linear analysis methods. 

There are [many](https://en.wikipedia.org/wiki/Positive-definite_kernel#Examples_of_p.d._kernels) positive definite kernels. Common kernels are the linear (dot product) kernel,
\begin{equation}
    k(\mathbf{x}, \mathbf{x}^{\prime}) = \mathbf{x}^T \mathbf{x},
\end{equation}
and the Gaussian kernel,
\begin{equation}
    k(\mathbf{x}, \mathbf{x}^{\prime}) = \exp{\left(-\gamma\lVert \mathbf{x} - \mathbf{x}^{\prime}\rVert^2\right)}.
\end{equation}

In this notebook we use the gaussian kernel (```rbf_kernel(XA, XB, gamma=1.0)```), however, we have also included the linear kernel function (```linear_kernel(XA, XB)```) in ```sklearn```. You can check rather easily that if you use a linear kernel all of the kernel methods reduce explicitly to linear regression or principal component analysis in the original feature space.

In [ ]:
# Changing the kernels in this cell will change them throughout the notebook tutorial.
# The function variable designates the kernel function to use throughout the notebook.
# The string variable is used to pass to the utility classes.
kernel_params = {"kernel": "rbf", "gamma": 1.0}
kernel_func = partial(rbf_kernel, gamma=1.0)
kernel_type = "gaussian"

The [representer theorem](https://en.wikipedia.org/wiki/Representer_theorem) guarantees that if the kernel is a positive definite function (both these two examples are) there is a Hilbert space with elements $\phi(\mathbf{x})$ whose dot product reproduces the kernel, i.e.

\begin{equation}
    k(\mathbf{x}, \mathbf{x}^{\prime}) = \phi(\mathbf{x})^T\phi(\mathbf{x}^{\prime}).
\end{equation}

This Hilbert space is known as the [**reproducing kernel Hilbert space**](https://en.wikipedia.org/wiki/Reproducing_kernel_Hilbert_space), or RKHS. If one builds a kernel matrix $\mathbf{K}$ whose elements contain the kernel computed between the corresponding pair of samples, 
For simple kernels, such as the linear kernel shown above, the RKHS is intuitive: $\phi(\mathbf{x}) = \mathbf{x}$.
However, for even slightly more complex kernel functions, the RKHS may be infinitely dimensional, such as the [RBF kernel](https://en.wikipedia.org/wiki/Radial_basis_function_kernel).
If one builds a kernel matrix $\mathbf{K}$ whose elements contain the kernel computed between the corresponding pair of samples, 

\begin{equation}
    K_{ij} = k(\mathbf{x}_i, \mathbf{x}_j)
\end{equation}

the formal relation with the reproducing features can be written in a matrix notation

\begin{equation}
\quad \mathbf{K} = \mathbf{\Phi_X\Phi_X^T}.
\end{equation}

$\mathbf{\Phi_X}$ indicates a matrix that holds the values of the RKHS features for each sample in $\mathbf{X}$, which means that $\mathbf{X}$and $\mathbf{\Phi_X}$ have the same number of samples, but different numbers of features. To simplify the notation, in this notebook we drop the $\mathbf{X}$ subscript and refer to this matrix simply as $\mathbf{\Phi}$.

In [ ]:
K_train_raw = kernel_func(X_train, X_train)

It is important to realize that even if in general $\phi(\mathbf{x})$ is not known, _for a given data set_ the representer theorem provides an explicit construction for an approximation of the RKHS features. The construction is very closely related to the KPCA method that we discuss below. 

Start by writing an eigendecomposition of the kernel matrix, 

\begin{equation}
    \mathbf{K} = \mathbf{U_K} \mathbf{\Lambda_K} \mathbf{U_K}^T.
\end{equation}

If one defines 
\begin{equation}
\mathbf{\Phi} =  \mathbf{U_K} \mathbf{\Lambda_K}^{1/2} = \mathbf{K} \mathbf{U_K} \mathbf{\Lambda_K}^{-1/2}
\end{equation}
one sees that $\mathbf{\Phi}\mathbf{\Phi}^T = \mathbf{K}$: the eigenvectors of the kernel matrix make it possible to construct a set of features whose scalar product reproduces exactly the values of the kernel for the dataset. 

The second equality is important because it provides a recipe to build an _approximate_ set of RKHS features for a _new_ set of points. If $\mathbf{K}_{NN}$ indicates the kernel matrix for the train set, and the matrix $\mathbf{K}_{N'N}$ contains the kernels between some new (e.g. test-set) samples and the train samples, which means the size of $\mathbf{K}_{N'N}$  is $n_{new\ samples}\times n_{train\ sample}$,  one can compute 
\begin{equation}
\mathbf{\Phi}_{N'N} = \mathbf{K}_{N'N} \mathbf{U} \mathbf{\Lambda}^{-1/2},
\end{equation}
where $\mathbf{U}$ and $\mathbf{\Lambda}$ refer to the eigendecomposition of the square $\mathbf{K}_{NN}$.
Then, $\mathbf{\Phi}_{N'N}$ is a matrix whose entries approximate the RKHS features for the new set of points. 

In [ ]:
v_K, U_K = np.linalg.eigh(K_train_raw)

# U_K/v_K are already sorted, but in *increasing* order, so reverse them
U_K = np.flip(U_K, axis=1)
v_K = np.flip(v_K, axis=0)

U_K = U_K[:, v_K > 0]
v_K = v_K[v_K > 0]

Phi = U_K @ np.diag(np.sqrt(v_K))

print(np.linalg.norm((K_train_raw - Phi @ Phi.T)))

## Kernel Centering

Just as it is often convenient to center features for linear methods, it is often beneficial to work with kernels that are also "centered". Centering a kernel can be understood in terms of centering of the associated RKHS features.  

Let us first introduce the centering matrix $\mathbf{1}_{N'N}$, which is just is a $N'\times N$ matrix for which each element takes value $1/N$. 

Using the expression above for $\mathbf{\Phi}$, one can compute 

\begin{equation}
\bar{\mathbf{\Phi}}_{N'N} = \mathbf{1}_{N'N} \mathbf{K}_{NN} \mathbf{U} \mathbf{\Lambda}^{-1/2}
\end{equation}

A similar centering matrix can be built for $\mathbf{\Phi}_{NN}$

\begin{equation}
\bar{\mathbf{\Phi}}_{NN} = \mathbf{1}_{NN} \mathbf{K}_{NN} \mathbf{U} \mathbf{\Lambda}^{-1/2}
\end{equation}

The centered kernel $\tilde{\mathbf{K}}_{N'N}$ reads

\begin{equation}
\tilde{\mathbf{K}}_{N'N} = (\mathbf{\Phi}_{N'N} - \bar{\mathbf{\Phi}}_{N'N})(\mathbf{\Phi}_{NN}-\bar{\mathbf{\Phi}}_{NN})^T = 
\mathbf{K}_{N'N} -  \mathbf{1}_{N'N} \mathbf{K}_{NN} -  \mathbf{K}_{N'N}\mathbf{1}_{NN}
+ \mathbf{1}_{N'N} \mathbf{K}_{NN} \mathbf{1}_{NN}.
\end{equation}

which can be computed without the need of ever evaluating explicitly the RKHS features.
This is a common pattern in kernel methods: RKHS features allow casting problems in a linear
language, but eventually the linear problem can yield an equation in which one only needs to evaluate the kernel function.

In this notebook, we center the kernels, which can be done with a utility function

In [ ]:
k_centerer = KernelNormalizer()
K_train = k_centerer.fit_transform(K_train_raw)
K_scale = k_centerer.scale_

## The Testing Set Kernel

For new data, we must generate a new kernel. Notice that a kernel takes *two* sets of data as arguments, one of which can be our testing $\mathbf{X}_{N'}$ and the other the training $\mathbf{X}_N$. This contains the kernels computed between all of the test set samples $N'$ and the train set samples $N$. This can be computed as 

In [ ]:
K_test = kernel_func(X_test, X_train)

We center with the training kernel as reference, as discussed above

In [ ]:
# centering relative to the approximate RHKS defined by
# the training kernel matrix can be achieved specifying
# K_train as the reference kernel matrix

K_test = k_centerer.transform(K_test)

# Kernel PCA (KPCA)

In [kernel principal component analysis](https://en.wikipedia.org/wiki/Kernel_principal_component_analysis)
(KPCA) we take advantage of the kernel in order to 
introduce a non-linear transformation of the feature space 
[(Scholkopf 1996)](http://www.face-rec.org/algorithms/Kernel/kernelPCA_scholkopf.pdf), 
[(Scholkopf 1998)](http://www.doi.org/10.1162/089976698300017467), and then proceed to single out the largest-variance directions in RKHS. 

With the linear (dot product) kernel, performing KPCA with a linear kernel is equivalent to performing standard PCA.

## The Principal Components

KPCA proceeds analogously to PCA. First, the eigenvalues and eigenvectors of $\mathbf{K}$ are computed:
\begin{equation}
    \mathbf{K} = \mathbf{U_K} \mathbf{\Lambda_K} \mathbf{U_K}^T
\end{equation}

In [ ]:
v_K, U_K = np.linalg.eigh(K_train)

# U_K/v_K are already sorted, but in *increasing* order, so reverse them
U_K = np.flip(U_K, axis=1)
v_K = np.flip(v_K, axis=0)

U_K = U_K[:, v_K > 0]
v_K = v_K[v_K > 0]

## The KPCA Projection

The KPCA projection may then be computed taking the first $n_{PCA}$ components 
\begin{equation}
\mathbf{T}=\mathbf{\hat{U}_K}\mathbf{\hat{\Lambda}_K}^{1/2} = \mathbf{\hat{U}_K}\mathbf{\hat{\Lambda}_K} (\mathbf{\hat{U}_K}^T\mathbf{\hat{U}_K}) \mathbf{\hat{\Lambda}_K}^{-1/2} = \mathbf{K} \mathbf{\hat{U}_K} \mathbf{\hat{\Lambda}_K}^{-1/2}
\end{equation}
\begin{equation}
    \mathbf{T} = \mathbf{K}\mathbf{P}_{KT} = \mathbf{K} \mathbf{\hat{U}_K} \mathbf{\hat{\Lambda}_K}^{-1/2}.
\end{equation}

**Remember that in PCA, the projection $\mathbf{T} = \mathbf{X}\mathbf{U}_C$. So why the factor of $\mathbf{\Lambda}_K^{-1/2}$?**

The eigenvectors of $\mathbf{K} = \mathbf{\Phi}\mathbf{\Phi}^T$ are in fact analogous to the eigenvectors of the Gram matrix $\mathbf{X}\mathbf{X}^T$ and thereby related to those of the covariance $\mathbf{C} = \mathbf{X}^T\mathbf{X} = \mathbf{U}_C \mathbf{\Lambda}_C \mathbf{U}_C^T$ by $\mathbf{X}\mathbf{U_C}$, which is not normalized. In essence, the $\mathbf{\Lambda}_K^{-1/2}$ factor serves to normalize our projection matrix. [(Tipping 2001)](https://papers.nips.cc/paper/1791-sparse-kernel-principal-component-analysis.pdf) 

Note also that the KPCA latent space corresponds to the highest-variance components in the RKHS: if one does not truncate the latent space, $\mathbf{T}=\mathbf{\Phi}$

In [ ]:
n_PC = 2

In [ ]:
PKT = U_K[:, :n_PC] @ np.diagflat(1.0 / np.sqrt(v_K[0:n_PC]))

The expression for the testing data is then identical to that for the projection of the train set:
\begin{equation}
    \mathbf{T} = \mathbf{K}_{N'N} \mathbf{P}_{KT}
\end{equation}
$\mathbf{P}_{KT}$ is the projection obtained during training, while the $\mathbf{K}_{N'N}$ indicates the kernel between the test set and the training points.

In [ ]:
T_KPCA_train = K_train @ PKT
T_KPCA_test = K_test @ PKT

**Note**: if we used a linear kernel, the projection would be identical to the PCA projection (modulo reflection, since the sign of the eigenvectors is not defined), and it would likewise correspond to the Classical MDS solution.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=dbl_fig)
plot_projection(
    Y_train, T_KPCA_train, fig=fig, ax=axes[0], title="KPCA of Training Data", **cmaps
)
plot_projection(
    Y_test, T_KPCA_test, fig=fig, ax=axes[1], title="KPCA of Testing Data", **cmaps
)
fig.subplots_adjust(wspace=0.4)

Even though this is not done very often (because the kernel acts in a different space than the original feature space) it is possible to reconstruct an approximation of the feature vector based on the KPCA latent space. Consider this like "predicting" $\mathbf{X}$, similar to predicting $\mathbf{Y}$ in linear regression. The projection matrix $\mathbf{P}_{TX}$ corresponds to the least-square weights
    
\begin{equation}    
\mathbf{P}_{TX} = (\mathbf{T}\mathbf{T}^T)^{-1}\mathbf{T}^T \mathbf{X} =  \mathbf{\hat{\Lambda}}_K^{-1} \mathbf{T}^T \mathbf{X}
\end{equation}

Where the factor of $\mathbf{\Lambda_K}^{-1}$ arises from the fact that
$\mathbf{T}^T\mathbf{T}=
\mathbf{\Lambda_K}^{-1/2}\mathbf{U_K}^T\mathbf{K}^T\mathbf{K}\mathbf{U_K}\mathbf{\Lambda_K}^{-1/2}=
\mathbf{\Lambda_K}^{1/2}\mathbf{U_K}^T\mathbf{U_K}\mathbf{\Lambda_K}^{1/2}=
\mathbf{\Lambda_K}$.

In [ ]:
PTX = np.diagflat(1 / ((v_K[:n_PC]))) @ T_KPCA_train.T @ X_train

Xr_train = T_KPCA_train @ PTX
Xr_test = T_KPCA_test @ PTX

The KPCA projection approximates the kernel matrix, so one can check for convergence using the [Frobenius norm](https://en.wikipedia.org/wiki/Matrix_norm#Frobenius_norm).

In [ ]:
K_approx_train = T_KPCA_train @ T_KPCA_train.T

K_test_test = kernel_func(X_test, X_test)
K_test_test = k_centerer.transform(K_test_test)
K_approx_test = T_KPCA_test @ T_KPCA_test.T

table_from_dict(
    [
        get_stats(
            x=X_train,
            xr=Xr_train,
            y=Y_train,
            t=T_KPCA_train,
            k=K_train,
            kapprox=K_approx_train,
        ),
        get_stats(
            x=X_test,
            xr=Xr_test,
            y=Y_test,
            t=T_KPCA_test,
            k=K_test_test,
            kapprox=K_approx_test,
        ),
    ],
    headers=["Training", "Testing"],
    title="KPCA",
)

# Implementation in `scikit-learn`

Classes from `sklearn` enable computing KPCA and KRR. Here we'll review the basics of their usage; readers are encouraged to refer to sklearn documentation for further detail.

In [ ]:
from sklearn.decomposition import KernelPCA
from sklearn.kernel_ridge import KernelRidge

## KPCA

In [ ]:
# We set kernel = 'precomputed' and supply kpca.fit(X) with X = K_train to improve computational efficiency.
kpca = KernelPCA(n_components=2, **kernel_params)

Calling `kpca.fit(K_train)` will generate the kernel for $\mathbf{X}$ and compute/internally store the eigenvectors/values.

In [ ]:
kpca.fit(K_train)

Calling `kpca.transform(K)` will compute and return the KPCA projection $\mathbf{T}$.

In [ ]:
T_KPCA_train = kpca.transform(K_train)
T_KPCA_test= kpca.transform(K_test)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=dbl_fig)

plot_projection(
    Y_train, T_KPCA_train, title="KPCA of Training Data", fig=fig, ax=axes[0], **cmaps
)
plot_projection(
    Y_test, T_KPCA_test, title="KPCA of Testing Data", fig=fig, ax=axes[1], **cmaps
)

fig.subplots_adjust(wspace=0.5)
plt.show()

In [ ]:
PTX = np.diagflat(1 / ((v_K[:n_PC]))) @ T_KPCA_train.T @ X_train

Xr_train = T_KPCA_train @ PTX
Xr_test = T_KPCA_test @ PTX

K_approx_train = T_KPCA_train @ T_KPCA_train.T
K_test_test = kernel_func(X_test, X_test)
K_approx_test = T_KPCA_test @ T_KPCA_test.T

table_from_dict(
    [
        get_stats(
            x=X_train,
            xr=Xr_train,
            y=Y_train,
            t=T_KPCA_train,
            k=K_train,
            kapprox=K_approx_train,
        ),
        get_stats(
            x=X_test,
            xr=Xr_test,
            y=Y_test,
            t=T_KPCA_test,
            k=K_test_test,
            kapprox=K_approx_test,
        ),
    ],
    headers=["Training", "Testing"],
    title="KPCA",
)